In [1]:
from sklearn import feature_extraction
from sklearn import linear_model
import numpy as np
import pandas as pd
from scipy.sparse import hstack

In [2]:
data = pd.read_csv("data/salary-train.csv")
data_test = pd.read_csv("data/salary-test-mini.csv")

In [9]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [11]:
data["FullDescription"] = data["FullDescription"].str.lower().replace('[^a-z0-9]', ' ', regex = True)

In [12]:
data_test["FullDescription"] = data_test["FullDescription"].str.lower().replace('[^a-z0-9]', ' ', regex = True)

In [13]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)
data_test['LocationNormalized'].fillna('nan', inplace=True)
data_test['ContractTime'].fillna('nan', inplace=True)

In [14]:
vector_dict = feature_extraction.DictVectorizer()
X_train_categ = vector_dict.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = vector_dict.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [24]:
data_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [25]:
vector_text = feature_extraction.text.TfidfVectorizer(min_df=5)
X_train_desc = vector_text.fit_transform(data["FullDescription"])
X_test_desc = vector_text.transform(data_test["FullDescription"])

In [27]:
X_test_desc.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [67]:
X_train = hstack([X_train_desc, X_train_categ])

In [68]:
X_test = hstack([X_test_desc, X_test_categ])

In [71]:
clf = linear_model.Ridge(alpha=1)
clf.fit(X_train, data["SalaryNormalized"])

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [74]:
predict = clf.predict(X_test)

In [75]:
predict

array([ 56574.4335807 ,  37206.23029832])

In [77]:
" ".join([str(round(s, 2)) for s in predict])

'56574.43 37206.23'